# **RAG with Memory:**

* **BaseChatMessageHistory**
* **InMemoryChatMessageHistory**
* **RunnableWithMessageHistory**
* **trim_messages**



**BaseChatMessageHistory:**<br>
<u>This is an **abstract base class** in LangChain that defines the blueprint for how chat message history should be managed.</u> It serves as the foundation for implementing various message history storage mechanisms.<br>
When creating custom chat history storage solutions, you extend this class and define how the messages are stored, retrieved, and cleared.<br>

**Key Features:**
* Provides a common interface for managing chat history.
* Can be extended to implement custom storage backends (e.g., databases, files, memory).


**Methods:**<br>
* **`add_user_message:`** Adds a user's message to the history.
* **`add_ai_message:`** Adds an AI-generated message to the history.
* **`clear:`** Clears the message history.



**InMemoryChatMessageHistory:**<br>
This is a **concrete implementation** of **`BaseChatMessageHistory`**, storing the chat history in memory. It is lightweight and simple to use for smaller applications or testing.<br>

**Key Features:**
* Stores all chat messages (user and AI) in a Python list.
* Suitable for short-lived conversations or applications where persistence isn't required.

**RunnableWithMessageHistory:**<br>
This is a **decorator-like utility** used to make a function or runnable object work with chat message history. <u>It allows you to combine existing message history with a callable object to process and generate outputs.</u><br>


**Key Features:**
* Facilitates the use of chat history with functional pipelines.
* Automatically integrates the chat history into callable objects.

In [ ]:
# install necessary libaries:

%pip install --upgrade --quiet sentence_transformers
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-google-genai langchain-chroma bs4

In [2]:
# Load the Tokens:

from google.colab import userdata
import os

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
HF_TOKEN = userdata.get('HF_TOKEN')

os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY
os.environ['HF_TOKEN'] = HF_TOKEN

## **Load Model/LLM:**

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-1-pro",
    temperature=0.4,
    max_tokens=512,
    timeout=None,
    max_retries=2
)

print(model.invoke("hi").content)

Hi there! How can I help you today?



In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(model.invoke("hi").content)

'Hi there! How can I help you today?\n'

## **Load Embeddings:**

In [ ]:
# Get the Embeddings:

from langchain.embeddings import HuggingFaceEmbeddings


model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

## **With Out Memory: QA Bot**

In [5]:
from langchain_core.messages import HumanMessage
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# QA Bot:

while True:
  message = input("Write your query:")
  if message == "bye":
    print("Good Bye have a great day!")
    break
  else:
    print(parser.invoke(model.invoke([HumanMessage(content=message)])))

Write your query:Hi!
Hi! How can I help you today?

Write your query:what is 2+2?
2 + 2 = 4

Write your query:which math question I ask before?
I have no memory of past conversations. Each interaction we have starts fresh.  If you'd like me to help with a math question, please ask it again.

Write your query:bye
Good Bye have a great day!


## **With Memory: Conversational ChatBot**

In [6]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [7]:
parser = StrOutputParser()

result = model.invoke(
    [
        HumanMessage(content="Hi! I'm cristinao"),
        AIMessage(content="Hello cristinao! How can I assist you today?"),
        HumanMessage(content="What's my name?")
    ]
)


parser.invoke(result)

'You said your name is cristinao.\n'

### **use Memory:**

In [8]:
# Use:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [9]:
# store the session information:

store={}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [10]:
# Model with Memory:

model_with_memory = RunnableWithMessageHistory(model, get_session_history)

#### **Chat based on Session:**
For each sesstion chat/conversation will store.

In [11]:
# Define Fast Sesstion id:
config = {"configurable": {"session_id": "firstchat"}}

In [12]:
# Chat with Fast session:

model_with_memory.invoke([HumanMessage(content="Hi! I'm Dibyedu Biswas, working as an AI/ML Engineer.")], config=config).content

"Hi Dibyedu Biswas! It's nice to meet you.  Is there anything I can help you with today?  Since you're an AI/ML Engineer, perhaps you have a question about a specific algorithm, a coding problem, or are looking for information on a particular AI/ML topic?  I'm ready to assist if you have something in mind.\n"

In [13]:
model_with_memory.invoke([HumanMessage(content="What is my name?")], config=config).content

'Your name is Dibyedu Biswas.\n'

In [14]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Dibyedu Biswas, working as an AI/ML Engineer.", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Dibyedu Biswas! It's nice to meet you.  Is there anything I can help you with today?  Since you're an AI/ML Engineer, perhaps you have a question about a specific algorithm, a coding problem, or are looking for information on a particular AI/ML topic?  I'm ready to assist if you have something in mind.\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-cb9be545-904b-40d7-b4f1-c40bd171c6db-0', usage_metadata={'input_tokens': 20, 'output_tokens': 79, 'total_tokens': 99, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Dibyedu Biswas.\n', additional_kwargs={}, response_metada

In [15]:
# Define Second Sesstion id:
config = {"configurable": {"session_id": "secondchat"}}

In [16]:
model_with_memory.invoke([HumanMessage(content="what is my name?")], config=config).content

"I have no access to information about you, including your name.  I'm a large language model, and my interactions are entirely within the current conversation.  I don't retain information between conversations.  You haven't told me your name, so I don't know it.\n"

#### **use prompt, without memory**

In [17]:
# Use The Prompting:

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


prompt=ChatPromptTemplate.from_messages(
    [
      ("system","You are a helpful assistant, named Lily. Answer all questions to the best of your ability.",),
      MessagesPlaceholder(variable_name="messages"),
    ]
)


chain = prompt | model


chain.invoke({"messages": ["hi! I'm bob"]})

AIMessage(content="Hi Bob! It's nice to meet you. How can I help you today?\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-4e92fef4-ce66-4451-8a29-c139ac9fa7f7-0', usage_metadata={'input_tokens': 26, 'output_tokens': 19, 'total_tokens': 45, 'input_token_details': {'cache_read': 0}})

In [18]:
chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]}).content

"Hi Bob! It's nice to meet you. How can I help you today?\n"

In [ ]:

chain.invoke({"messages": [HumanMessage(content="what is my name")]}).content

"Unfortunately, I don't have access to that information. I can only see what you type in our current conversation.  I have no way of knowing your name unless you tell me.\n"

#### **use prompt, memory and store memory in a session:**

In [19]:
config = {"configurable": {"session_id": "thirdchat"}}

model_with_memory=RunnableWithMessageHistory(chain, get_session_history)

In [20]:
response = model_with_memory.invoke(
    [HumanMessage(content="Hi! I'm Jim"),
     ],config=config # {"session_id": "thirdchat"}
)
response.content

"Hi Jim, it's nice to meet you! How can I help you today?\n"

In [21]:
response=model_with_memory.invoke(
    [HumanMessage(content="hi what is my name"),
     ],config=config # {"session_id": "thirdchat"}
)
print(response.content)

You told me your name is Jim.



In [22]:
response=model_with_memory.invoke(
    [HumanMessage(content="what is 2+2?"),
     ],config=config # {"session_id": "thirdchat"}
)
print(response.content)

2 + 2 = 4



In [23]:
response=model_with_memory.invoke(
    [HumanMessage(content="who is a indian cricket team caption?"),
     ],config=config # {"session_id": "thirdchat"}
)
print(response.content)

The current captain of the Indian cricket team varies depending on the format (Test, ODI, T20I).  It's best to check a reliable sports news source for the most up-to-date information, as captaincy can change frequently.



In [25]:
response=model_with_memory.invoke(
    [HumanMessage(content="what should i add in previous addition so that i will become 10?"),
     ],config=config # {"session_id": "thirdchat"}
)
print(response.content)

You previously added 2 + 2 = 4.  To get to 10, you need to add 6 more. (10 - 4 = 6)



In [26]:
response=model_with_memory.invoke(
    [HumanMessage(content="what is my name?"),
     ],config=config # {"session_id": "thirdchat"}
)
print(response.content)

You told me your name is Jim.



In [27]:
# get the session information:

store.keys()

dict_keys(['firstchat', 'secondchat', 'thirdchat'])

In [28]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Dibyedu Biswas, working as an AI/ML Engineer.", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Dibyedu Biswas! It's nice to meet you.  Is there anything I can help you with today?  Since you're an AI/ML Engineer, perhaps you have a question about a specific algorithm, a coding problem, or are looking for information on a particular AI/ML topic?  I'm ready to assist if you have something in mind.\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-cb9be545-904b-40d7-b4f1-c40bd171c6db-0', usage_metadata={'input_tokens': 20, 'output_tokens': 79, 'total_tokens': 99, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Dibyedu Biswas.\n', additional_kwargs={}, response_metada

### **Trimming the Conversation:**

In [29]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage, trim_messages

In [34]:
trimmer = trim_messages(
    max_tokens=30, # store the 60 tokens
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [67]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [36]:
# Get the number of tokens:

model.get_num_tokens_from_messages(messages)
# It means, we have 51 tokens during the previous conversation or messages or chat history.

51

In [38]:
# triming the messages or conversation:

trimmed_message = trimmer.invoke(messages)
trimmed_message

[HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [39]:
# Get the number of tokens:

model.get_num_tokens_from_messages(trimmed_message)

29

In [51]:
# Add messages:
trimmed_message.extend([HumanMessage(content="what's my name?")])


In [52]:
trimmed_message

[HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={})]

In [53]:
trimmed_message.extend([AIMessage(content="My Name is Dibyendu Biswas.")])
trimmed_message

[HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='My Name is Dibyendu Biswas.', additional_kwargs={}, response_metadata={})]

In [68]:
messages

[HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [64]:
prompt = ChatPromptTemplate.from_messages(
    [
      ("system","You are a helpful assistant, named Lily. Answer all questions to the best of your ability.",),
      MessagesPlaceholder(variable_name="messages"),
    ]
)


from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough


chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

In [69]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I have no way of knowing your name. I haven't been given any information about you personally.  If you'd like to tell me, I'd be happy to use it!\n"

In [46]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

"You haven't asked me a math problem yet.  Is there one you'd like me to solve?\n"

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [60]:

model_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [61]:
# Define the fiftchat session:
config = {"configurable": {"session_id": "fifthchat"}}

In [70]:
response = model_with_memory.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I have no way of knowing your name. I'm an AI and don't have access to any personal information about you unless you tell me.\n"

In [71]:
response = model_with_memory.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You haven't actually asked a math problem yet.  Please tell me the math problem you'd like me to solve or help you with.\n"

In [72]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Dibyedu Biswas, working as an AI/ML Engineer.", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Dibyedu Biswas! It's nice to meet you.  Is there anything I can help you with today?  Since you're an AI/ML Engineer, perhaps you have a question about a specific algorithm, a coding problem, or are looking for information on a particular AI/ML topic?  I'm ready to assist if you have something in mind.\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-cb9be545-904b-40d7-b4f1-c40bd171c6db-0', usage_metadata={'input_tokens': 20, 'output_tokens': 79, 'total_tokens': 99, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Dibyedu Biswas.\n', additional_kwargs={}, response_metada